<a href="https://colab.research.google.com/github/toby-j/fyp_concept/blob/master/pytorchIG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>